In [ ]:
# NOTE:
# This installation cell is required only when running the notebook in cloud
# environments such as Google Colab. Local environments with pre-installed
# dependencies can safely skip this step.
!pip install -q kaggle xgboost openpyxl


In [ ]:
# =========================
# SECTION 1: Imports & Globals (with Kaggle)
# =========================
import os, glob, random, json, shutil, zipfile, subprocess, sys, time
from pathlib import Path
from typing import List, Tuple, Dict

import numpy as np
import pandas as pd

import cv2
from PIL import Image

import torch
from torch import nn
from torchvision import transforms, models
from torchvision.models import MobileNet_V2_Weights

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, confusion_matrix, RocCurveDisplay
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

import matplotlib.pyplot as plt
import seaborn as sns  # optional; keep only if used later

# ==== General settings ====
SEED = 42
FIXED_SEED = True
N_RUNS = 5
TEST_SIZE = 0.30

NUM_FRAMES_PER_VIDEO = 64
FRAME_SIZE = 224
BATCH_SIZE = 128
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# IMPORTANT (Reviewer-safety note):
# Training supervision is VIDEO-LEVEL only (Normal/Anomalous per video).
# No frame-level annotations are used for training. Any segment/frame scores
# (if computed later) are inference-based and must not be interpreted as
# supervised temporal localization.

# ==== Paths ====
# Colab default root is /content. For local runs you can set:
# export WSAD_ROOT=/path/to/your/workdir
ROOT_DIR = os.environ.get("WSAD_ROOT", "/content")

DATA_DIR     = os.path.join(ROOT_DIR, "ucf_crime")   # contains Normal/ and Anomaly/
FRAMES_DIR   = os.path.join(ROOT_DIR, "frames")
FEATURES_DIR = os.path.join(ROOT_DIR, "features")
RESULTS_DIR  = os.path.join(ROOT_DIR, "results")
CACHE_DIR    = os.path.join(ROOT_DIR, "cache")

for d in [DATA_DIR, FRAMES_DIR, FEATURES_DIR, RESULTS_DIR, CACHE_DIR]:
    os.makedirs(d, exist_ok=True)

# ==== Kaggle config ====
# Update this to your Kaggle dataset slug (owner/dataset-slug)
KAGGLE_DATASET   = "owner/ucf-crime-videos"  # <-- replace with the correct slug
KAGGLE_DEST      = DATA_DIR
KAGGLE_JSON_PATH = "/root/.kaggle/kaggle.json"  # default path for Kaggle API token (Colab)

# ==== Experiments ====
EXPERIMENTS = [
    {"name": "baseline",         "context": False, "aug": False, "mmd": False, "attention": False},
    {"name": "context",          "context": True,  "aug": False, "mmd": False, "attention": False},
    {"name": "aug",              "context": False, "aug": True,  "mmd": False, "attention": False},
    {"name": "context_aug_mmd",  "context": True,  "aug": True,  "mmd": True,  "attention": False},
    {"name": "attention_mil",    "context": False, "aug": False, "mmd": False, "attention": True},
]
# Note: 'attention_mil' is an experimental extension; the core baseline here is mean-pooling MIL.

print(f"Using DEVICE = {DEVICE}")
print(f"ROOT_DIR = {ROOT_DIR}")


In [ ]:
# =========================
# SECTION 2: Reproducibility helpers
# =========================
# Ensures reproducibility across runs (CPU/GPU, NumPy, PyTorch)
def set_seed(seed: int = 42):
    import os, random
    import numpy as np
    import torch
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)

set_seed(SEED)


In [ ]:
# =========================
# SECTION 3: Kaggle API helpers + Download (robust)
# =========================
# SECURITY NOTE:
# Do NOT commit kaggle.json to GitHub. Keep it local/Colab only.

import subprocess, shlex

def _in_colab():
    try:
        import google.colab  # noqa: F401
        return True
    except Exception:
        return False

def ensure_kaggle_api(kaggle_json_path=KAGGLE_JSON_PATH):
    """
    Ensures kaggle.json exists and is configured correctly.
    - Colab: allows one-time upload if missing.
    - Local: place kaggle.json at the provided path and rerun.
    """
    # Allow overriding via environment variable (useful for local runs)
    kaggle_json_path = os.environ.get("KAGGLE_JSON", kaggle_json_path)

    if not os.path.exists(kaggle_json_path):
        if _in_colab():
            from google.colab import files
            print("[KAGGLE] kaggle.json not found. Please upload it now…")
            files.upload()
            if not os.path.exists("kaggle.json"):
                raise FileNotFoundError("Upload cancelled. Please provide kaggle.json.")
            os.makedirs("/root/.kaggle", exist_ok=True)
            shutil.move("kaggle.json", "/root/.kaggle/kaggle.json")
            kaggle_json_path = "/root/.kaggle/kaggle.json"
        else:
            raise FileNotFoundError(
                f"kaggle.json not found at {kaggle_json_path}. "
                "Set env KAGGLE_JSON=/path/to/kaggle.json or place it at KAGGLE_JSON_PATH."
            )

    # Default Kaggle config path (works in Colab/Linux)
    os.makedirs("/root/.kaggle", exist_ok=True)
    if kaggle_json_path != "/root/.kaggle/kaggle.json":
        shutil.copy(kaggle_json_path, "/root/.kaggle/kaggle.json")
    os.chmod("/root/.kaggle/kaggle.json", 0o600)
    print("[KAGGLE] API is configured.")

def _run(cmd: str):
    """Runs a shell command and prints output."""
    print(f"$ {cmd}")
    result = subprocess.run(
        shlex.split(cmd),
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )
    print(result.stdout)
    if result.returncode != 0:
        raise RuntimeError(f"Command failed: {cmd}")

def kaggle_list_files(dataset_slug=KAGGLE_DATASET):
    """Lists dataset files on Kaggle (diagnostic)."""
    _run(f"kaggle datasets files -d {dataset_slug}")

def kaggle_download_dataset(dataset_slug=KAGGLE_DATASET, dest_dir=KAGGLE_DEST, force=False):
    """
    Downloads dataset contents into dest_dir.
    If force=True, re-downloads even if files exist.
    """
    os.makedirs(dest_dir, exist_ok=True)
    cmd = f"kaggle datasets download -d {dataset_slug} -p {dest_dir}"
    if force:
        cmd += " --force"
    _run(cmd)

    # Unzip any downloaded zip files
    zips = glob.glob(os.path.join(dest_dir, "*.zip"))
    if not zips:
        print("[WARN] No zip files found after download — maybe already unzipped.")
    for z in zips:
        print(f"[KAGGLE] Unzipping: {os.path.basename(z)}")
        with zipfile.ZipFile(z, "r") as zf:
            zf.extractall(dest_dir)
    print("[KAGGLE] Download & unzip complete.")

def normalize_ucf_structure(root=DATA_DIR):
    """
    Heuristic structure normalization:
      DATA_DIR/Normal/*.mp4
      DATA_DIR/Anomaly/*.mp4
    NOTE: This relies on path/name keywords (normal/anomaly). Verify results.
    """
    mp4s = glob.glob(os.path.join(root, "**", "*.mp4"), recursive=True)
    if not mp4s:
        print("[WARN] No .mp4 files found yet in DATA_DIR.")
        return

    norm_dir = os.path.join(root, "Normal")
    anom_dir = os.path.join(root, "Anomaly")
    os.makedirs(norm_dir, exist_ok=True)
    os.makedirs(anom_dir, exist_ok=True)

    moved = 0
    for p in mp4s:
        base = os.path.basename(p)
        low = p.lower()
        if "normal" in low:
            dst = os.path.join(norm_dir, base)
        elif "anomaly" in low or "abnormal" in low or "anomal" in low:
            dst = os.path.join(anom_dir, base)
        else:
            continue
        if os.path.abspath(p) != os.path.abspath(dst):
            os.makedirs(os.path.dirname(dst), exist_ok=True)
            shutil.move(p, dst)
            moved += 1

    # Cleanup empty folders
    for dirpath, dirnames, filenames in os.walk(root, topdown=False):
        if dirpath in [norm_dir, anom_dir, root]:
            continue
        if not os.listdir(dirpath):
            shutil.rmtree(dirpath, ignore_errors=True)

    print(f"[STRUCTURE] Moved {moved} files into Normal/ and Anomaly/.")

def ensure_ucf_from_kaggle(force=False, list_files=False):
    """
    Full workflow: configure API -> (optional) list files -> download -> unzip -> normalize structure.
    """
    ensure_kaggle_api()
    if list_files:
        kaggle_list_files()
    kaggle_download_dataset(force=force)
    normalize_ucf_structure(DATA_DIR)
    n_normal = len(glob.glob(os.path.join(DATA_DIR, "Normal", "*.mp4")))
    n_anom   = len(glob.glob(os.path.join(DATA_DIR, "Anomaly", "*.mp4")))
    print(f"[READY] Normal: {n_normal} | Anomaly: {n_anom} (root: {DATA_DIR})")


In [ ]:
# =========================
# SECTION 3.1 — Kaggle API (Colab-simple)
# =========================
# SECURITY NOTE:
# Do NOT commit kaggle.json to GitHub. Keep it local/Colab only.

def setup_kaggle_colab_simple():
    """Colab-only helper to upload kaggle.json and configure Kaggle API."""
    try:
        from google.colab import files  # type: ignore
    except Exception:
        print("[INFO] Not running in Colab. Skip this cell and configure Kaggle API locally.")
        print("       Option 1: place kaggle.json at /root/.kaggle/kaggle.json (Linux/Colab)")
        print("       Option 2: use ensure_kaggle_api() from SECTION 3 (robust)")
        return

    import os, shutil

    print("⬆️ Please upload kaggle.json …")
    uploaded = files.upload()

    if "kaggle.json" not in uploaded:
        raise FileNotFoundError("kaggle.json was not uploaded. Please retry.")

    os.makedirs("/root/.kaggle", exist_ok=True)
    shutil.move("kaggle.json", "/root/.kaggle/kaggle.json")
    os.chmod("/root/.kaggle/kaggle.json", 0o600)

    print("✅ Kaggle API configured for Colab.")

# Run only when needed (Colab)
setup_kaggle_colab_simple()


In [ ]:
# =========================
# SECTION 3.2 — Download + Unzip + Normalize (Colab-simple)
# =========================
# NOTE (Dataset / Licensing):
# UCF-Crime is not distributed with this repository. Downloading datasets from Kaggle
# depends on the dataset uploader's terms and may not reflect the original dataset license.
# Use at your own responsibility and ensure compliance with applicable terms.

import os, glob, zipfile, shutil

# 1) Dataset slug (override via environment variable if needed)
KAGGLE_DATASET = os.environ.get("KAGGLE_DATASET", "minhajuddinmeraj/anomalydetectiondatasetucf")
DATA_DIR       = os.environ.get("WSAD_DATA_DIR", "/content/ucf_crime")
os.makedirs(DATA_DIR, exist_ok=True)

# 2) (Optional) list available files (diagnostic)
print("[INFO] Listing Kaggle dataset files (optional)...")
!kaggle datasets files -d $KAGGLE_DATASET

# 3) Download dataset
print(f"📥 Downloading {KAGGLE_DATASET} → {DATA_DIR}")
!kaggle datasets download -d $KAGGLE_DATASET -p $DATA_DIR --force

# 4) Unzip any ZIP files
zips = glob.glob(os.path.join(DATA_DIR, "*.zip"))
if not zips:
    print("[WARN] No ZIP files found (maybe already unzipped).")
else:
    for z in zips:
        print(f"🗜️ Unzipping: {os.path.basename(z)}")
        with zipfile.ZipFile(z, "r") as zf:
            zf.extractall(DATA_DIR)

# 5) Normalize structure into Normal/ and Anomaly/
def normalize_ucf_structure(root=DATA_DIR):
    """
    Organizes mp4 files into:
      root/Normal/*.mp4
      root/Anomaly/*.mp4

    NOTE: This is a heuristic that assumes any path/name containing 'normal'
    belongs to Normal, and everything else belongs to Anomaly. Verify the dataset
    structure and adjust if needed.
    """
    norm_dir = os.path.join(root, "Normal")
    anom_dir = os.path.join(root, "Anomaly")
    os.makedirs(norm_dir, exist_ok=True)
    os.makedirs(anom_dir, exist_ok=True)

    mp4s = glob.glob(os.path.join(root, "**", "*.mp4"), recursive=True)
    if not mp4s:
        print("[WARN] No .mp4 files found. Please inspect the extracted contents.")
        return

    moved = 0
    for p in mp4s:
        base = os.path.basename(p)
        low  = p.lower()
        if "normal" in low:
            dst = os.path.join(norm_dir, base)
        else:
            dst = os.path.join(anom_dir, base)
        if os.path.abspath(p) != os.path.abspath(dst):
            shutil.move(p, dst)
            moved += 1

    # Cleanup empty folders
    for dirpath, _, __ in os.walk(root, topdown=False):
        if dirpath in [norm_dir, anom_dir, root]:
            continue
        if not os.listdir(dirpath):
            shutil.rmtree(dirpath, ignore_errors=True)

    print(f"📁 Organized: moved {moved} files → Normal/ & Anomaly/")

normalize_ucf_structure(DATA_DIR)

# 6) Final counts
n_normal = len(glob.glob(os.path.join(DATA_DIR, "Normal", "*.mp4")))
n_anom   = len(glob.glob(os.path.join(DATA_DIR, "Anomaly", "*.mp4")))
print(f"✅ READY — Normal: {n_normal} | Anomaly: {n_anom} | root: {DATA_DIR}")


In [ ]:
# =========================
# SECTION 4 — Normalize structure & scan videos
# =========================
import os, glob, shutil
import pandas as pd

# NOTE:
# This section builds VIDEO-LEVEL metadata (one label per video).
# Ensure DATA_DIR is defined in SECTION 1 (do not redefine it here).

LABEL_MAP = {"Normal": 0, "Anomaly": 1}

def normalize_ucf_structure(root: str = DATA_DIR, dry_run: bool = False) -> None:
    """
    Ensures the standard structure:
      root/Normal/*.mp4
      root/Anomaly/*.mp4

    NOTE (Heuristic):
    This normalization uses a simple rule:
      if 'normal' in file path/name -> Normal
      else -> Anomaly
    Verify the resulting folders if you use a different dataset packaging.
    """
    norm_dir = os.path.join(root, "Normal")
    anom_dir = os.path.join(root, "Anomaly")
    os.makedirs(norm_dir, exist_ok=True)
    os.makedirs(anom_dir, exist_ok=True)

    mp4s = glob.glob(os.path.join(root, "**", "*.mp4"), recursive=True)
    if not mp4s:
        print("[WARN] No .mp4 files found. Ensure download/unzip is completed.")
        return

    moved = 0
    planned = 0
    for p in mp4s:
        base = os.path.basename(p)
        low  = p.lower()

        dst = os.path.join(norm_dir, base) if "normal" in low else os.path.join(anom_dir, base)

        if os.path.abspath(p) != os.path.abspath(dst):
            planned += 1
            if dry_run:
                continue
            shutil.move(p, dst)
            moved += 1

    # Cleanup empty folders
    if not dry_run:
        for dirpath, _, __ in os.walk(root, topdown=False):
            if dirpath in {root, norm_dir, anom_dir}:
                continue
            if not os.listdir(dirpath):
                shutil.rmtree(dirpath, ignore_errors=True)

    if dry_run:
        print(f"[DRY RUN] Would move {planned} files into Normal/ & Anomaly/.")
    else:
        print(f"📁 Organized: moved {moved} files → Normal/ & Anomaly/")

def scan_videos(data_dir: str = DATA_DIR) -> pd.DataFrame:
    """
    Builds a VIDEO-LEVEL metadata DataFrame with:
      video_id, label, path, cls
    """
    rows = []
    for cls in ["Normal", "Anomaly"]:
        cls_dir = os.path.join(data_dir, cls)
        if not os.path.isdir(cls_dir):
            print(f"[WARN] Missing class dir: {cls_dir}")
            continue

        for p in glob.glob(os.path.join(cls_dir, "*.mp4")):
            vid = os.path.splitext(os.path.basename(p))[0]
            rows.append({
                "video_id": vid,
                "label": LABEL_MAP[cls],
                "path": p,
                "cls": cls
            })

    meta = pd.DataFrame(rows)
    if meta.empty:
        print("[WARN] No videos detected after normalization.")
    else:
        n_norm = int((meta["label"] == 0).sum())
        n_anom = int((meta["label"] == 1).sum())
        print(f"✅ READY — Videos: {len(meta)} | Normal: {n_norm} | Anomaly: {n_anom}")
    return meta

# === Run SECTION 4 ===
normalize_ucf_structure(DATA_DIR, dry_run=False)
meta = scan_videos(DATA_DIR)
assert not meta.empty, "No videos detected after normalization."


In [ ]:
# =========================
# SECTION 5.1 — Frame Extraction helpers
# =========================
import os, glob, time
import cv2
from PIL import Image
import numpy as np
import pandas as pd
from typing import Optional

# Depends on earlier sections:
# DATA_DIR, FRAMES_DIR, NUM_FRAMES_PER_VIDEO

os.makedirs(FRAMES_DIR, exist_ok=True)

# NOTE: Frames are sampled uniformly across the video for feature extraction.
# This is not intended for precise temporal anomaly localization.

def extract_frames_for_video(video_path: str, out_dir: str, n_frames: int = NUM_FRAMES_PER_VIDEO) -> int:
    """
    Extracts n_frames uniformly across the video and saves JPGs to out_dir.
    Returns the number of newly saved images (does not recount existing ones).
    """
    os.makedirs(out_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        cap.release()
        return 0

    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total <= 0:
        cap.release()
        return 0

    existing = sorted(glob.glob(os.path.join(out_dir, "*.jpg")))
    if len(existing) >= n_frames:
        cap.release()
        return 0

    idxs = np.linspace(0, total - 1, num=n_frames, dtype=int)
    saved = 0
    for i, fi in enumerate(idxs):
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(fi))
        ok, frame = cap.read()
        if not ok:
            continue
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        out_path = os.path.join(out_dir, f"{i:04d}.jpg")
        Image.fromarray(frame_rgb).save(out_path, quality=92)
        saved += 1

    cap.release()
    return saved

def ensure_frames(
    meta: pd.DataFrame,
    n_frames: int = NUM_FRAMES_PER_VIDEO,
    limit_per_class: Optional[int] = None
) -> dict:
    """
    Extracts frames for each video in `meta` if needed.
    limit_per_class: e.g., 2 to run only first N videos per class for quick tests.
    """
    stats = {"videos_processed": 0, "new_images_saved": 0, "normal_done": 0, "anomaly_done": 0}
    class_count = {"Normal": 0, "Anomaly": 0}

    t0 = time.time()
    for _, r in meta.iterrows():
        cls = r["cls"]
        if limit_per_class is not None and class_count[cls] >= limit_per_class:
            continue

        vid_dir = os.path.join(FRAMES_DIR, cls, r["video_id"])
        saved_now = extract_frames_for_video(r["path"], vid_dir, n_frames=n_frames)

        stats["videos_processed"] += 1
        stats["new_images_saved"] += saved_now
        class_count[cls] += 1

        if cls == "Normal":
            stats["normal_done"] += 1
        else:
            stats["anomaly_done"] += 1

    dt = time.time() - t0
    print(
        f"✅ ensure_frames → videos: {stats['videos_processed']}, new_jpg: {stats['new_images_saved']}, "
        f"Normal:{stats['normal_done']}, Anomaly:{stats['anomaly_done']} | time: {dt:.1f}s"
    )
    return stats


In [ ]:
# =========================
# SECTION 5.2 — Run full frame extraction (no limits)
# =========================
import glob

assert 'meta' in globals() and not meta.empty, "meta غير موجودة: شغّل SECTION 4 أولاً."

# Safety switch (recommended for GitHub users)
RUN_FULL_EXTRACTION = True  # set to False for quick tests

if RUN_FULL_EXTRACTION:
    _ = ensure_frames(meta, n_frames=NUM_FRAMES_PER_VIDEO, limit_per_class=None)

    n_frames_normal  = len(glob.glob(os.path.join(FRAMES_DIR, "Normal",  "*", "*.jpg")))
    n_frames_anomaly = len(glob.glob(os.path.join(FRAMES_DIR, "Anomaly", "*", "*.jpg")))
    print(f"✅ Frames saved → Normal:{n_frames_normal} | Anomaly:{n_frames_anomaly} | root:{FRAMES_DIR}")
else:
    print("[SKIP] Full frame extraction is disabled. Set RUN_FULL_EXTRACTION=True to run.")


In [ ]:
# =========================
# SECTION 6 — Transforms (BASE / AUG)
# =========================
from torchvision import transforms

# NOTE:
# FRAME_SIZE is defined in SECTION 1. Do not redefine it here to avoid inconsistencies.

BASE_TRANSFORM = transforms.Compose([
    transforms.Resize((FRAME_SIZE, FRAME_SIZE)),
    transforms.ToTensor(),  # [0..1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225]),
])

# Used only when the experiment configuration enables augmentation (aug=True).
AUG_TRANSFORM = transforms.Compose([
    transforms.Resize((FRAME_SIZE, FRAME_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([
        transforms.ColorJitter(brightness=0.2, contrast=0.2,
                               saturation=0.2, hue=0.05)
    ], p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225]),
])


In [ ]:
# =========================
# SECTION 7 — Feature Extraction (MobileNetV2) + Safe Cache
# =========================
import os, glob, numpy as np, pandas as pd
from PIL import Image
import torch
from torch import nn
from torchvision import models
from torchvision.models import MobileNet_V2_Weights
from typing import Optional, Set, Dict

# Required from earlier sections:
# FRAMES_DIR, FEATURES_DIR, DEVICE, BATCH_SIZE, BASE_TRANSFORM, AUG_TRANSFORM

os.makedirs(FEATURES_DIR, exist_ok=True)

# -------------------------------------------------------------
# 7.1 — MobileNetV2 as 1280D feature extractor
# -------------------------------------------------------------
_mobilenet = models.mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT).features
FEATURE_EXTRACTOR = nn.Sequential(
    _mobilenet,
    nn.AdaptiveAvgPool2d((1, 1))
).to(DEVICE).eval()

for p in FEATURE_EXTRACTOR.parameters():
    p.requires_grad = False

# -------------------------------------------------------------
# 7.2 — Safe batch inference (with fallback on CUDA OOM)
# -------------------------------------------------------------
@torch.no_grad()
def _infer_batch(img_tensors, batch_size: Optional[int] = None) -> np.ndarray:
    if batch_size is None:
        batch_size = len(img_tensors)
    feats_list, i = [], 0

    while i < len(img_tensors):
        chunk = img_tensors[i:i+batch_size]
        try:
            xb = torch.stack(chunk, dim=0).to(DEVICE)
            fb = FEATURE_EXTRACTOR(xb).reshape(len(chunk), -1)
            feats_list.append(fb.cpu().numpy().astype(np.float32))
            i += batch_size
        except RuntimeError as e:
            # Only attempt batch reduction on CUDA OOM
            if ("CUDA" in str(e) or "out of memory" in str(e).lower()) and (DEVICE.type == "cuda") and batch_size > 1:
                torch.cuda.empty_cache()
                batch_size = max(1, batch_size // 2)
                continue
            raise

    return np.concatenate(feats_list, axis=0) if feats_list else np.empty((0, 1280), np.float32)

# -------------------------------------------------------------
# 7.3 — Context features (global mean/std per frame)
# -------------------------------------------------------------
def _context_feat(pil_img: Image.Image) -> np.ndarray:
    arr = np.asarray(pil_img).astype(np.float32) / 255.0
    return np.array([arr.mean(), arr.std()], dtype=np.float32)

# -------------------------------------------------------------
# 7.4 — Extract features for a full video (with optional context)
# -------------------------------------------------------------
def extract_video_features(frame_paths: list, transform, add_context: bool = False) -> np.ndarray:
    """
    Converts a list of frame paths to features of shape (num_frames, D)
    where D = 1280 (or 1282 if add_context=True).
    """
    feats, batch_imgs, pending_ctx = [], [], []

    for p in frame_paths:
        try:
            with Image.open(p) as img:
                img = img.convert("RGB")
                x = transform(img)
                batch_imgs.append(x)
                if add_context:
                    pending_ctx.append(_context_feat(img))
        except Exception:
            continue

        if len(batch_imgs) == BATCH_SIZE:
            fb = _infer_batch(batch_imgs)
            if add_context:
                fb = np.hstack([fb, np.asarray(pending_ctx, np.float32)])
            feats.append(fb)
            batch_imgs, pending_ctx = [], []

    if batch_imgs:
        fb = _infer_batch(batch_imgs)
        if add_context:
            fb = np.hstack([fb, np.asarray(pending_ctx, np.float32)])
        feats.append(fb)

    if not feats:
        D = 1282 if add_context else 1280
        return np.empty((0, D), dtype=np.float32)

    return np.concatenate(feats, axis=0).astype(np.float32)

# -------------------------------------------------------------
# 7.5 — Safe feature file path per experiment/run/video
# -------------------------------------------------------------
def _feature_npz_path(cls: str, video_id: str, add_context: bool, use_aug: bool, run_id: Optional[int]) -> str:
    exp_tag = ("ctx" if add_context else "base") + "_" + ("aug" if use_aug else "noaug")
    run_tag = f"run{run_id}" if (use_aug and run_id is not None) else "run0"
    out_dir = os.path.join(FEATURES_DIR, exp_tag, run_tag, cls)
    os.makedirs(out_dir, exist_ok=True)
    return os.path.join(out_dir, f"{video_id}.npz")

# -------------------------------------------------------------
# 7.6 — Build safe feature cache (no leakage)
# -------------------------------------------------------------
def build_feature_cache_for_split(
    frames_df: pd.DataFrame,                 # columns: video_id, cls
    add_context: bool,
    train_only_aug: bool,
    train_video_ids: Optional[Set[str]],
    run_id: int = 0,
) -> Dict[str, str]:
    """
    Builds a safe feature cache:
      - add_context: appends (mean,std) per frame.
      - train_only_aug: applies AUG only to training videos.
      - run_id: used to separate cached outputs per run.
    """
    id2npz: Dict[str, str] = {}

    for _, r in frames_df.iterrows():
        vid, cls = r["video_id"], r["cls"]
        frame_paths = sorted(glob.glob(os.path.join(FRAMES_DIR, cls, vid, "*.jpg")))
        if not frame_paths:
            continue

        use_aug = bool(train_only_aug and (train_video_ids is not None) and (vid in train_video_ids))
        out_npz = _feature_npz_path(cls, vid, add_context=add_context, use_aug=use_aug, run_id=run_id)

        if os.path.exists(out_npz):
            id2npz[vid] = out_npz
            continue

        tfm = AUG_TRANSFORM if use_aug else BASE_TRANSFORM
        feats = extract_video_features(frame_paths, transform=tfm, add_context=add_context)
        label = 0 if cls.lower() == "normal" else 1
        feat_dim = feats.shape[1] if feats.ndim == 2 and feats.size else (1282 if add_context else 1280)

        np.savez_compressed(
            out_npz,
            features=feats,
            video_id=vid,
            cls=cls,
            label=np.int64(label),
            feat_dim=np.int32(feat_dim),
            add_context=np.int8(1 if add_context else 0),
            used_aug=np.int8(1 if use_aug else 0),
            run_id=np.int32(run_id),
        )
        id2npz[vid] = out_npz

    print(f"✅ Cached feature files: {len(id2npz)} | context={add_context} | aug_train_only={train_only_aug}")
    return id2npz


In [ ]:
# === Smoke Test: build base_noaug/run0 cache for videos that have extracted frames ===
import glob
import pandas as pd

assert 'meta' in globals() and not meta.empty, "شغّل Section 4 أولًا."
assert 'FRAMES_DIR' in globals() and 'FEATURES_DIR' in globals(), "شغّل Section 1 أولًا (paths)."

frames_rows = []
missing = 0

for _, r in meta.iterrows():
    img_dir = os.path.join(FRAMES_DIR, r["cls"], r["video_id"])
    imgs = glob.glob(os.path.join(img_dir, "*.jpg"))
    if imgs:
        frames_rows.append({"video_id": r["video_id"], "cls": r["cls"]})
    else:
        missing += 1

frames_df = pd.DataFrame(frames_rows).drop_duplicates(subset=["video_id"])
print(f"videos in meta: {len(meta)} | videos with frames: {len(frames_df)} | missing frames: {missing}")

assert not frames_df.empty, (
    "frames_df is empty. Run Section 5.2 to extract frames first, or check FRAMES_DIR path."
)

id2npz_smoke = build_feature_cache_for_split(
    frames_df=frames_df,
    add_context=False,        # base features only
    train_only_aug=False,     # no augmentation
    train_video_ids=None,
    run_id=0
)

print(f"✅ cached feature files: {len(id2npz_smoke)}")


In [ ]:
# =========================
# SECTION 8 — Load bags + mean pooling + audits
# =========================
import os, numpy as np, pandas as pd
from typing import Optional, Set, Tuple, List

def load_bags_from_cache(
    frames_df: pd.DataFrame,
    add_context: bool,
    train_only_aug: bool,
    train_video_ids: Optional[Set[str]],
    run_id: int
):
    """
    Loads per-video frame features from cached NPZ files and returns:
      - bags: list of arrays (n_frames, D)
      - labels: (N,) int64
      - vids: list of video_id
      - pooled: (N, D) float32 mean-pooled representations
    """
    bags: List[np.ndarray] = []
    labels: List[int] = []
    vids: List[str] = []
    pooled: List[np.ndarray] = []

    for _, r in frames_df.iterrows():
        vid, cls = r["video_id"], r["cls"]

        # Apply AUG only for training videos if enabled
        use_aug = bool(train_only_aug and (train_video_ids is not None) and (vid in train_video_ids))

        # Use the same path logic as Section 7 to avoid mismatch
        npz_path = _feature_npz_path(cls, vid, add_context=add_context, use_aug=use_aug, run_id=run_id)

        if not os.path.exists(npz_path):
            continue

        d = np.load(npz_path, allow_pickle=False)
        Xv = d["features"]  # (n_frames, D)

        # Skip empty feature arrays (prevents NaNs in pooling)
        if Xv.ndim != 2 or Xv.shape[0] == 0:
            continue

        bags.append(Xv.astype(np.float32))
        labels.append(int(d["label"]) if "label" in d.files else (0 if cls == "Normal" else 1))
        vids.append(vid)
        pooled.append(Xv.mean(axis=0).astype(np.float32))

    if not bags:
        raise RuntimeError("لم يتم تحميل أي حقائب من الكاش. تأكد أن Section 7 نجح وأن ملفات NPZ موجودة.")

    return bags, np.array(labels, dtype=np.int64), vids, np.vstack(pooled).astype(np.float32)


def audit_no_aug_on_test(
    test_indices,
    vids: List[str],
    add_context: bool,
    train_only_aug: bool,
    train_ids: Optional[Set[str]],
    run_id: int
) -> bool:
    """
    Ensures no augmentation was used for test videos.
    Reads used_aug from NPZ metadata.
    """

    # If augmentation is not enabled for training, the audit is trivially satisfied.
    if not train_only_aug or train_ids is None:
        print("[AUDIT] train_only_aug disabled or train_ids=None → skipping no-aug-on-test audit.")
        return True

    for i in test_indices:
        vid = vids[i]

        # Test video must not be in training IDs
        assert vid not in train_ids, f"[AUDIT] Split issue: test video {vid} appears in train_ids."

        found = False
        used_aug = 0

        # We don't know cls here reliably; search both
        for cls_name in ("Normal", "Anomaly"):
            # For augmented train videos: exp_tag ends with _aug and run{run_id}
            path_aug = _feature_npz_path(cls_name, vid, add_context=add_context, use_aug=True, run_id=run_id)
            # For non-aug videos: exp_tag ends with _noaug and run0 (per Section 7 logic)
            path_noaug = _feature_npz_path(cls_name, vid, add_context=add_context, use_aug=False, run_id=run_id)

            for p in (path_aug, path_noaug):
                if os.path.exists(p):
                    d = np.load(p, allow_pickle=False)
                    used_aug = int(d["used_aug"]) if "used_aug" in d.files else 0
                    found = True
                    break
            if found:
                break

        assert found, f"[AUDIT] Feature file not found for video {vid}."
        assert used_aug == 0, f"[LEAK] Test video {vid} has used_aug=1 (augmentation leakage)."

    print("[AUDIT] Passed: no augmentation applied to test videos.")
    return True


In [ ]:
# =========================
# SECTION 9 — Attention MIL (optional)
# =========================
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

class BagsDataset(Dataset):
    def __init__(self, bags, labels):
        self.bags = bags
        self.labels = labels
    def __len__(self):
        return len(self.bags)
    def __getitem__(self, i):
        return self.bags[i], int(self.labels[i])

def _collate_single(batch):
    bag_np, y = batch[0]
    bag = torch.tensor(bag_np, dtype=torch.float32)        # (T,D)
    y   = torch.tensor([y], dtype=torch.float32)           # (1,)
    return bag, y

class AttentionMILNet(nn.Module):
    def __init__(self, in_dim, hidden=256):
        super().__init__()
        self.attn = nn.Sequential(
            nn.Linear(in_dim, hidden), nn.Tanh(),
            nn.Linear(hidden, 1)
        )
        self.cls  = nn.Sequential(
            nn.Linear(in_dim, hidden), nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, bag, return_attention: bool = False):
        if bag.dim() == 3:
            bag = bag.squeeze(0)
        w = self.attn(bag)                      # (T,1)
        a = torch.softmax(w, dim=0)             # (T,1)
        pooled = torch.sum(a * bag, dim=0)      # (D,)
        logit  = self.cls(pooled).view(1)       # (1,)
        if return_attention:
            return logit, a.squeeze(-1)         # (T,)
        return logit

def evaluate_attention_mil_once(
    train_bags, train_y, test_bags, test_y,
    epochs=5, lr=1e-3, batch_size=1, seed=42, device=DEVICE
):
    # basic safety checks
    assert len(train_bags) > 0 and train_bags[0].ndim == 2, "train_bags must be a non-empty list of (T,D) arrays."

    # seeds
    np.random.seed(seed)
    torch.manual_seed(seed)
    if device.type == "cuda":
        torch.cuda.manual_seed_all(seed)

    in_dim = train_bags[0].shape[1]
    model  = AttentionMILNet(in_dim).to(device)
    opt    = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    loss_fn= nn.BCEWithLogitsLoss()

    tr_ds  = BagsDataset(train_bags, train_y)
    te_ds  = BagsDataset(test_bags,  test_y)
    tr_dl  = DataLoader(tr_ds, batch_size=1, shuffle=True,  collate_fn=_collate_single)
    te_dl  = DataLoader(te_ds, batch_size=1, shuffle=False, collate_fn=_collate_single)

    model.train()
    for _ in range(epochs):
        for xb, yb in tr_dl:
            xb = xb.to(device)
            yb = yb.to(device)
            opt.zero_grad()
            logit = model(xb)                  # (1,)
            loss = loss_fn(logit, yb)          # shapes match
            loss.backward()
            opt.step()

    model.eval()
    probs, preds, ys = [], [], []
    with torch.no_grad():
        for xb, yb in te_dl:
            xb = xb.to(device)
            logit = model(xb)
            p = torch.sigmoid(logit).item()
            probs.append(p)
            preds.append(1 if p >= 0.5 else 0)
            ys.append(int(yb.item()))

    from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
    y_true = np.array(ys); y_pred = np.array(preds); y_prob = np.array(probs)
    acc = float(accuracy_score(y_true, y_pred))
    f1  = float(f1_score(y_true, y_pred))
    auc = float('nan') if len(np.unique(y_true)) < 2 else float(roc_auc_score(y_true, y_prob))
    return {"accuracy": acc, "f1": f1, "auc": auc, "y_true": y_true, "y_pred": y_pred, "y_prob": y_prob}


In [ ]:
# =========================
# SANITY CHECK — frames vs cached features
# =========================
assert "frames_df" in globals(), "frames_df غير موجودة — شغّل Sections 4 و5 أولاً."
assert "FEATURES_DIR" in globals(), "FEATURES_DIR غير معرّف."

import glob, os

print("frames_df rows:", len(frames_df))
npz_files = glob.glob(os.path.join(FEATURES_DIR, "**", "*.npz"), recursive=True)
print("NPZ count:", len(npz_files))

if len(npz_files) < len(frames_df):
    print("⚠️ WARNING: Some videos do NOT have cached features yet.")
else:
    print("✅ All videos have at least one cached feature file.")


In [ ]:
# =========================
# SECTION 10 — Splits + classic models (VIDEO-level)
# =========================
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

def stratified_group_shuffle_splits(labels, groups, test_size=0.30, n_splits=5, random_state=42):
    """
    Group-aware train/test splits with approximate class balance.
    IMPORTANT: groups should be unique per sample (use video_id) to avoid leakage.
    """
    labels = np.asarray(labels)
    groups = np.asarray(groups)
    idx = np.arange(len(labels))

    # Safety: avoid group leakage
    assert len(np.unique(groups)) == len(groups), (
        "Group leakage risk: groups must be unique per sample. Use video_id as groups."
    )

    cls0 = idx[labels == 0]
    cls1 = idx[labels == 1]

    for k in range(n_splits):
        rs = random_state + k
        gss0 = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=rs)
        gss1 = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=rs)

        tr0, te0 = next(gss0.split(cls0, groups=groups[cls0]))
        tr1, te1 = next(gss1.split(cls1, groups=groups[cls1]))

        train_idx = np.concatenate([cls0[tr0], cls1[tr1]])
        test_idx  = np.concatenate([cls0[te0], cls1[te1]])

        yield np.sort(train_idx), np.sort(test_idx)

def get_models_pipelines(seed=42):
    base = [("scaler", StandardScaler())]
    return {
        "Random Forest": Pipeline(base + [("clf", RandomForestClassifier(
            n_estimators=200, random_state=seed, n_jobs=-1
        ))]),
        "SVM": Pipeline(base + [("clf", SVC(
            probability=True, random_state=seed
        ))]),
        "KNN": Pipeline(base + [("clf", KNeighborsClassifier(
            n_neighbors=5
        ))]),
        "Logistic Regression": Pipeline(base + [("clf", LogisticRegression(
            max_iter=2000, random_state=seed
        ))]),
        "XGBoost": Pipeline(base + [("clf", XGBClassifier(
            n_estimators=300,
            eval_metric="logloss",
            random_state=seed,
            n_jobs=-1
        ))]),
    }

def eval_classic_once(pooled, labels, train_idx, test_idx, seed=42):
    """
    Train/evaluate classic models on ONE split.
    Metrics are VIDEO-level (one sample per video).
    """
    X_train, X_test = pooled[train_idx], pooled[test_idx]
    y_train, y_test = labels[train_idx], labels[test_idx]

    results = {}
    for name, pipe in get_models_pipelines(seed).items():
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)

        # IMPORTANT: use the PIPELINE for proba/decision (includes scaler)
        if hasattr(pipe, "predict_proba"):
            y_prob = pipe.predict_proba(X_test)[:, 1]
        elif hasattr(pipe, "decision_function"):
            df = pipe.decision_function(X_test)
            y_prob = (df - df.min()) / (df.max() - df.min() + 1e-12)
        else:
            y_prob = y_pred.astype(float)

        acc = float(accuracy_score(y_test, y_pred))
        f1  = float(f1_score(y_test, y_pred))
        auc = float("nan") if len(np.unique(y_test)) < 2 else float(roc_auc_score(y_test, y_prob))

        results[name] = {"accuracy": acc, "f1": f1, "auc": auc}

    return results


In [ ]:
# =========================
# SECTION 11 — Run all experiments (5× repeats, 70/30) — SAFE
# =========================
import os, json, glob
import numpy as np
import pandas as pd

EXPERIMENTS = [
    {"name":"baseline",         "context":False, "aug":False, "mmd":False, "attention":False},
    {"name":"context",          "context":True,  "aug":False, "mmd":False, "attention":False},
    {"name":"aug",              "context":False, "aug":True,  "mmd":False, "attention":False},
    {"name":"context_aug_mmd",  "context":True,  "aug":True,  "mmd":True,  "attention":False},
    {"name":"attention_mil",    "context":False, "aug":False, "mmd":False, "attention":True},
]

def export_mean_std(all_runs, exp_name, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    models = sorted({m for r in all_runs for m in r.keys()})
    rows = []

    for m in models:
        accs = [r[m]["accuracy"] for r in all_runs if m in r]
        f1s  = [r[m]["f1"] for r in all_runs if m in r]
        aucs = [r[m]["auc"] for r in all_runs if m in r and not np.isnan(r[m]["auc"])]

        mu = lambda xs: float(np.mean(xs)) if xs else float("nan")
        sd = lambda xs: float(np.std(xs, ddof=1)) if len(xs) > 1 else 0.0

        rows.append({
            "Model": m,
            "Accuracy_mean": mu(accs), "Accuracy_std": sd(accs),
            "F1_mean":       mu(f1s),  "F1_std":       sd(f1s),
            "AUC_mean":      mu(aucs), "AUC_std":      sd(aucs),
            "Runs": len(accs)
        })

    out_csv = os.path.join(out_dir, f"summary_{exp_name}_mean_std.csv")
    pd.DataFrame(rows).to_csv(out_csv, index=False)
    print(f"✅ saved summary: {out_csv}")

def run_all_experiments(frames_df: pd.DataFrame, meta_df: pd.DataFrame, n_runs=5, seed=42):
    """
    VIDEO-level evaluation:
      - classic models on mean-pooled video embeddings
      - attention MIL produces one score per video (not frame-level)
    """
    assert "video_id" in frames_df.columns and "cls" in frames_df.columns, "frames_df must have ['video_id','cls']"
    assert "video_id" in meta_df.columns and "label" in meta_df.columns, "meta_df must have ['video_id','label']"

    # groups MUST be unique per sample (video_id)
    groups = frames_df["video_id"].astype(str).values
    assert len(np.unique(groups)) == len(groups), "Group leakage risk: frames_df video_id must be unique per row."

    # Build labels safely (single index)
    meta_idx = meta_df.set_index("video_id")["label"]
    labels = meta_idx.reindex(frames_df["video_id"]).astype("float").values
    assert not np.isnan(labels).any(), "Some video_ids in frames_df are missing in meta_df."
    labels = labels.astype(np.int64)

    for exp in EXPERIMENTS:
        name = exp["name"]
        print(f"\n========== EXPERIMENT: {name} ==========")

        exp_root = os.path.join(RESULTS_DIR, name)
        os.makedirs(exp_root, exist_ok=True)

        # Save config for documentation
        with open(os.path.join(exp_root, "config.json"), "w") as f:
            json.dump({
                "experiment": name,
                "n_runs": n_runs,
                "test_size": 0.30,
                "seed": seed,
                "context": exp["context"],
                "aug_train_only": exp["aug"],
                "attention": exp["attention"],
                "note": "VIDEO-level evaluation (not frame-level AUC)."
            }, f, indent=2)

        all_runs = []

        for run_id, (tr_idx, te_idx) in enumerate(
            stratified_group_shuffle_splits(labels, groups, test_size=0.30, n_splits=n_runs, random_state=seed)
        ):
            run_dir = os.path.join(exp_root, f"run_{run_id}")
            os.makedirs(run_dir, exist_ok=True)

            train_vids = frames_df.iloc[tr_idx]["video_id"].astype(str).tolist()
            test_vids  = frames_df.iloc[te_idx]["video_id"].astype(str).tolist()

            pd.DataFrame({"video_id": train_vids, "split": "train"}).to_csv(os.path.join(run_dir, "split_train.csv"), index=False)
            pd.DataFrame({"video_id": test_vids,  "split": "test"} ).to_csv(os.path.join(run_dir, "split_test.csv"),  index=False)

            _overlap = set(train_vids).intersection(set(test_vids))
            assert not _overlap, f"Leak: overlap between splits: {len(_overlap)} vids"

            train_ids = set(train_vids)

            # Build / load feature cache (AUG only for train videos if enabled)
            _ = build_feature_cache_for_split(
                frames_df=frames_df,
                add_context=exp["context"],
                train_only_aug=exp["aug"],
                train_video_ids=train_ids,
                run_id=run_id
            )

            # Load bags + pooled
            bags, y, vids, pooled = load_bags_from_cache(
                frames_df=frames_df,
                add_context=exp["context"],
                train_only_aug=exp["aug"],
                train_video_ids=train_ids,
                run_id=run_id
            )

            vid2idx = {v: i for i, v in enumerate(vids)}
            tr = np.array([vid2idx[v] for v in train_vids if v in vid2idx], dtype=int)
            te = np.array([vid2idx[v] for v in test_vids  if v in vid2idx], dtype=int)

            # Guard: ensure split still has samples
            if len(tr) < 2 or len(te) < 2:
                print(f"[WARN] run {run_id}: too few cached samples (train={len(tr)}, test={len(te)}). Skipping.")
                continue

            # Audit: no augmentation on test
            audit_no_aug_on_test(te, vids, exp["context"], exp["aug"], train_ids, run_id)

            # Run evaluation
            if exp["attention"]:
                res = evaluate_attention_mil_once(
                    train_bags=[bags[i] for i in tr], train_y=y[tr],
                    test_bags=[bags[i] for i in te],  test_y=y[te],
                    epochs=5, lr=1e-3, batch_size=1, seed=seed+run_id, device=DEVICE
                )
                all_runs.append({"Attention-MIL": {"accuracy": res["accuracy"], "f1": res["f1"], "auc": res["auc"]}})
            else:
                res = eval_classic_once(pooled, y, tr, te, seed+run_id)
                all_runs.append(res)

            # Optional: MMD logging (diagnostic only)
            if exp["mmd"]:
                X0 = pooled[y == 0]
                X1 = pooled[y == 1]
                if len(X0) > 1 and len(X1) > 1:
                    from sklearn.metrics import pairwise_distances
                    from sklearn.metrics.pairwise import rbf_kernel

                    D = pairwise_distances(np.vstack([X0, X1]))
                    med = np.median(D[D > 0])
                    gamma = 1.0 / (2.0 * (med**2 + 1e-12))

                    mmd_val = (
                        rbf_kernel(X0, X0, gamma=gamma).mean() +
                        rbf_kernel(X1, X1, gamma=gamma).mean() -
                        2.0 * rbf_kernel(X0, X1, gamma=gamma).mean()
                    )
                    with open(os.path.join(run_dir, "mmd.txt"), "w") as f:
                        f.write(f"MMD: {float(mmd_val):.6f}\n")

        if not all_runs:
            print(f"[WARN] No successful runs for experiment '{name}'. Check frame extraction / feature cache.")
            continue

        export_mean_std(all_runs, name, exp_root)


In [ ]:
assert 'frames_df' in globals() and not frames_df.empty
assert 'meta' in globals() and not meta.empty

# التشغيل الكامل لكل التجارب مع 5 تكرارات
run_all_experiments(frames_df=frames_df, meta_df=meta, n_runs=5, seed=42)


In [ ]:
# قياس RF/SVM/KNN/LR/XGB على نفس التقسيمات المستخدمة في attention_mil (SAFE)
import os, glob
import numpy as np
import pandas as pd

from sklearn.base import clone
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

assert "RESULTS_DIR" in globals() and "FEATURES_DIR" in globals(), "RESULTS_DIR/FEATURES_DIR غير معرّفين"
assert "frames_df" in globals() and not frames_df.empty, "frames_df غير جاهز"
assert "meta" in globals() and not meta.empty, "meta غير جاهز"

ATT_DIR = os.path.join(RESULTS_DIR, "attention_mil")
assert os.path.isdir(ATT_DIR), f"attention_mil folder not found: {ATT_DIR}"

# نبني بيانات كلاسيكية (mean-pooled) من نفس كاش base_noaug/run0 (لأن attention_mil عندك بدون aug وبدون context)
X, y, vids = [], [], []

missing_npz = 0
empty_feats = 0

for _, r in frames_df.iterrows():
    vid = str(r["video_id"])
    cls = str(r["cls"])
    p = os.path.join(FEATURES_DIR, "base_noaug", "run0", cls, f"{vid}.npz")

    if not os.path.exists(p):
        missing_npz += 1
        continue

    d = np.load(p, allow_pickle=True)
    feats = d["features"]

    if feats.size == 0:
        empty_feats += 1
        continue

    X.append(feats.mean(axis=0).astype(np.float32))

    if "label" in d.files:
        y.append(int(d["label"]))
    else:
        cls_npz = d["cls"].item() if hasattr(d["cls"], "item") else str(d["cls"])
        y.append(0 if str(cls_npz).lower() == "normal" else 1)

    vids.append(vid)

X = np.asarray(X, dtype=np.float32)
y = np.asarray(y, dtype=np.int64)
vid2idx = {v: i for i, v in enumerate(vids)}

print(f"[INFO] pooled samples: {len(vids)} | missing_npz: {missing_npz} | empty_feats: {empty_feats}")

# نماذج (سنستخدم clone لكل run)
base_models = {
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1),
    "SVM": SVC(probability=True, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Logistic Regression": LogisticRegression(max_iter=2000, random_state=42),
    "XGBoost": XGBClassifier(
        random_state=42,
        n_estimators=300,
        n_jobs=-1,
        eval_metric="logloss"
    ),
}

all_runs = {m: [] for m in base_models.keys()}

# امشِ على نفس الـ runs التي مولدها attention_mil
run_dirs = sorted(glob.glob(os.path.join(ATT_DIR, "run_*")))
assert len(run_dirs) > 0, f"No run_* dirs found inside: {ATT_DIR}"

skipped_runs = 0

for rdir in run_dirs:
    tr_path = os.path.join(rdir, "split_train.csv")
    te_path = os.path.join(rdir, "split_test.csv")
    if not (os.path.exists(tr_path) and os.path.exists(te_path)):
        skipped_runs += 1
        continue

    tr_vids = pd.read_csv(tr_path)["video_id"].astype(str).tolist()
    te_vids = pd.read_csv(te_path)["video_id"].astype(str).tolist()

    tr_idx = np.array([vid2idx[v] for v in tr_vids if v in vid2idx], dtype=int)
    te_idx = np.array([vid2idx[v] for v in te_vids if v in vid2idx], dtype=int)

    if len(tr_idx) < 2 or len(te_idx) < 2:
        skipped_runs += 1
        continue

    scaler = StandardScaler()
    Xtr = scaler.fit_transform(X[tr_idx])
    Xte = scaler.transform(X[te_idx])
    ytr, yte = y[tr_idx], y[te_idx]

    for name, base_clf in base_models.items():
        clf = clone(base_clf)  # مهم: موديل جديد لكل run
        clf.fit(Xtr, ytr)
        yp = clf.predict(Xte)

        if hasattr(clf, "predict_proba"):
            prob = clf.predict_proba(Xte)[:, 1]
        elif hasattr(clf, "decision_function"):
            s = clf.decision_function(Xte)
            prob = (s - s.min()) / (s.max() - s.min() + 1e-12)
        else:
            prob = yp.astype(float)

        acc = float(accuracy_score(yte, yp))
        f1  = float(f1_score(yte, yp))
        auc = float("nan") if len(np.unique(yte)) < 2 else float(roc_auc_score(yte, prob))

        all_runs[name].append({"accuracy": acc, "f1": f1, "auc": auc})

print(f"[INFO] skipped_runs: {skipped_runs} / {len(run_dirs)}")

# احسب mean±std لكل نموذج واحفظ جدول مقارنة بجانب attention_mil
rows = []
for name, runs in all_runs.items():
    accs = [r["accuracy"] for r in runs]
    f1s  = [r["f1"] for r in runs]
    aucs = [r["auc"] for r in runs if not np.isnan(r["auc"])]

    mu = lambda a: float(np.mean(a)) if len(a) else float("nan")
    sd = lambda a: float(np.std(a, ddof=1)) if len(a) > 1 else 0.0

    rows.append({
        "Model": name,
        "Accuracy_mean": mu(accs), "Accuracy_std": sd(accs),
        "F1_mean":       mu(f1s),  "F1_std":       sd(f1s),
        "AUC_mean":      mu(aucs), "AUC_std":      sd(aucs),
        "Runs": len(runs)
    })

df = pd.DataFrame(rows).sort_values("AUC_mean", ascending=False)
out_csv = os.path.join(ATT_DIR, "summary_attention_vs_classic_5.csv")
df.to_csv(out_csv, index=False)
print("✅ saved:", out_csv)
display(df)


In [ ]:
# ---------- اجعل تحميل الملفات مرنًا (SAFE) ----------
import os, glob, zipfile
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

BASE_RESULTS = "/content/results"
PLOTS_DIR = os.path.join(BASE_RESULTS, "plots_summary")
os.makedirs(PLOTS_DIR, exist_ok=True)

def find_latest(pattern, exclude_dir="plots_summary"):
    """يبحث بشكل recursive داخل results/ ويعيد أحدث ملف يطابق النمط (حسب mtime)."""
    matches = glob.glob(os.path.join(BASE_RESULTS, "**", pattern), recursive=True)
    matches = [m for m in matches if exclude_dir not in m.replace("\\", "/")]
    if not matches:
        raise FileNotFoundError(f"لم أجد أي ملف يطابق النمط: {pattern}")
    matches.sort(key=lambda p: os.path.getmtime(p), reverse=True)
    print("→ using latest:", matches[0])
    return matches[0]

# اختَر أحدث ملفات الملخصات
classic_path = find_latest("summary_attention_vs_classic_5*.csv")
attn_path    = find_latest("summary_attention_mil_mean_std*.csv")

df_classic = pd.read_csv(classic_path)
df_attn    = pd.read_csv(attn_path)

# ضمان وجود عمود Model في ملف Attention
if "Model" not in df_attn.columns:
    df_attn.insert(0, "Model", "Attention-MIL")

# توحيد الأعمدة المطلوبة
needed = ["Model", "Accuracy_mean", "F1_mean", "AUC_mean",
          "Accuracy_std", "F1_std", "AUC_std", "Runs"]

for col in needed:
    if col not in df_classic.columns:
        df_classic[col] = pd.NA
    if col not in df_attn.columns:
        df_attn[col] = pd.NA

df_all = pd.concat([df_classic[needed], df_attn[needed]], ignore_index=True)

# تحويل الأعمدة الرقمية (لتفادي نصوص داخل CSV)
for c in ["Accuracy_mean","F1_mean","AUC_mean","Accuracy_std","F1_std","AUC_std","Runs"]:
    df_all[c] = pd.to_numeric(df_all[c], errors="coerce")

# حفظ جدول ملخص موحد
out_csv = os.path.join(PLOTS_DIR, "All_Models_Summary.csv")
df_all.to_csv(out_csv, index=False)
print("✅ saved unified table:", out_csv)

# --------- رسومات المقارنة (Accuracy / F1 / AUC) ----------
metrics = ["Accuracy_mean", "F1_mean", "AUC_mean"]
for metric in metrics:
    df_plot = df_all.dropna(subset=[metric]).copy()
    if df_plot.empty:
        print(f"[WARN] لا يوجد بيانات لرسم {metric}")
        continue

    # ترتيب الموديلات حسب أفضل قيمة في هذا المقياس
    order = df_plot.sort_values(metric, ascending=False)["Model"].tolist()

    plt.figure(figsize=(9,5))
    sns.barplot(x="Model", y=metric, data=df_plot, order=order)
    plt.title(f"{metric} Comparison Across Models (VIDEO-level)")
    plt.xticks(rotation=35, ha="right")
    plt.tight_layout()
    out_png = os.path.join(PLOTS_DIR, f"{metric}_comparison.png")
    plt.savefig(out_png, dpi=200)
    plt.close()
    print("✅ saved:", out_png)

# --------- ضغط كل الرسومات والنتائج في ملف ZIP ----------
zip_path = "/content/All_Results_and_Plots.zip"
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
    # ملفات الملخصات الأصلية
    zf.write(classic_path, os.path.relpath(classic_path, BASE_RESULTS))
    zf.write(attn_path,    os.path.relpath(attn_path,    BASE_RESULTS))

    # ملف الملخص الموحد
    zf.write(out_csv, os.path.relpath(out_csv, BASE_RESULTS))

    # مجلد الرسومات
    for root, _, files in os.walk(PLOTS_DIR):
        for f in files:
            fp = os.path.join(root, f)
            zf.write(fp, os.path.relpath(fp, BASE_RESULTS))

print("✅ جاهز:", zip_path)


In [ ]:
from google.colab import files
files.download("/content/All_Results_and_Plots.zip")


In [ ]:
# ===============================================
# SECTION: Collect all experiments results + plots into one ZIP (SAFE + MEANINGFUL)
# ===============================================
import os, glob, zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

BASE_RESULTS = "/content/results"
ALL_PLOTS_DIR = os.path.join(BASE_RESULTS, "All_Experiments_Plots")
os.makedirs(ALL_PLOTS_DIR, exist_ok=True)

# ---------- 1) اجمع فقط ملفات summary الخاصة بالتجارب (baseline/context/...) ----------
# نستثني أي summary داخل مجلدات التجميع نفسها لتفادي إعادة الالتقاط
summary_files = glob.glob(os.path.join(BASE_RESULTS, "*", "summary_*_mean_std.csv"))
print(f"📂 Found {len(summary_files)} per-experiment summary files")

dfs = []
for f in summary_files:
    exp_name = os.path.basename(os.path.dirname(f))  # اسم التجربة = اسم المجلد
    df = pd.read_csv(f)
    if "Model" not in df.columns:
        # لو لسبب ما ملف ملخص مختلف
        continue
    df.insert(0, "Experiment", exp_name)
    dfs.append(df)

if not dfs:
    raise FileNotFoundError("⚠️ لم أجد summary_*_mean_std.csv داخل مجلدات التجارب مباشرة.")

df_all = pd.concat(dfs, ignore_index=True)

# تأكد من الأعمدة الرقمية
for c in ["Accuracy_mean","F1_mean","AUC_mean","Accuracy_std","F1_std","AUC_std","Runs"]:
    if c in df_all.columns:
        df_all[c] = pd.to_numeric(df_all[c], errors="coerce")

df_all_path = os.path.join(ALL_PLOTS_DIR, "All_Experiments_Summary_RAW.csv")
df_all.to_csv(df_all_path, index=False)
print(f"✅ RAW combined summary saved → {df_all_path}")

# ---------- 2) خيار A: أفضل موديل لكل تجربة (حسب AUC_mean) ----------
best_per_exp = (
    df_all.dropna(subset=["AUC_mean"])
         .sort_values(["Experiment","AUC_mean"], ascending=[True, False])
         .groupby("Experiment", as_index=False)
         .head(1)
         .reset_index(drop=True)
)

best_path = os.path.join(ALL_PLOTS_DIR, "All_Experiments_BEST_Model.csv")
best_per_exp.to_csv(best_path, index=False)
print(f"✅ BEST-per-experiment table saved → {best_path}")

# ---------- 3) خيار B: مقارنة على موديل محدد (اختياري) ----------
TARGET_MODEL = None  # ضع مثلاً "Logistic Regression" أو "SVM" أو اتركها None لتعطيل هذا الخيار

if TARGET_MODEL is not None:
    df_target = df_all[df_all["Model"].astype(str) == TARGET_MODEL].copy()
    target_path = os.path.join(ALL_PLOTS_DIR, f"All_Experiments_{TARGET_MODEL.replace(' ','_')}.csv")
    df_target.to_csv(target_path, index=False)
    print(f"✅ Target-model table saved → {target_path}")
else:
    df_target = pd.DataFrame()

# ---------- 4) الرسومات: ارسم BEST-per-experiment (منطقي) ----------
metrics = ["Accuracy_mean", "F1_mean", "AUC_mean"]
for metric in metrics:
    if metric not in best_per_exp.columns:
        continue
    df_plot = best_per_exp.dropna(subset=[metric]).copy()
    if df_plot.empty:
        continue

    # ترتيب التجارب تنازليًا
    order = df_plot.sort_values(metric, ascending=False)["Experiment"].tolist()

    plt.figure(figsize=(10,5))
    sns.barplot(x="Experiment", y=metric, data=df_plot, order=order)
    plt.title(f"{metric} — Best Model per Experiment (VIDEO-level)")
    plt.xticks(rotation=35, ha="right")
    plt.tight_layout()
    out_png = os.path.join(ALL_PLOTS_DIR, f"{metric}_BEST_per_experiment.png")
    plt.savefig(out_png, dpi=200)
    plt.close()
    print("📊 saved:", out_png)

# (اختياري) لو اخترت TARGET_MODEL ارسمه كذلك
if not df_target.empty:
    for metric in metrics:
        df_plot = df_target.dropna(subset=[metric]).copy()
        if df_plot.empty:
            continue
        order = df_plot.sort_values(metric, ascending=False)["Experiment"].tolist()
        plt.figure(figsize=(10,5))
        sns.barplot(x="Experiment", y=metric, data=df_plot, order=order)
        plt.title(f"{metric} — {TARGET_MODEL} Across Experiments (VIDEO-level)")
        plt.xticks(rotation=35, ha="right")
        plt.tight_layout()
        out_png = os.path.join(ALL_PLOTS_DIR, f"{metric}_{TARGET_MODEL.replace(' ','_')}_per_experiment.png")
        plt.savefig(out_png, dpi=200)
        plt.close()
        print("📊 saved:", out_png)

# ---------- 5) ZIP: اضف كل CSV/PNG/TXT مع الحفاظ على البنية ----------
zip_path = "/content/All_Experiments_Results_and_Plots.zip"
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(BASE_RESULTS):
        for file in files:
            if file.endswith((".csv", ".png", ".txt")):
                fpath = os.path.join(root, file)
                zipf.write(fpath, os.path.relpath(fpath, BASE_RESULTS))

print(f"✅ كل التجارب محفوظة ومضغوطة هنا:\n{zip_path}")


In [ ]:
from google.colab import files
files.download("/content/All_Experiments_Results_and_Plots.zip")
